# Time Evolution & 2D Fields of PV Tendency Coefficients

Demonstrates:
1. **Time evolution** of β (intensification), αx / αy (propagation), γ (wave breaking / deformation)
2. **2D full-field maps** of each projected component at selected time steps

Uses synthetic data simulating a blocking event lifecycle: **onset → peak → decay**.

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.colors import TwoSlopeNorm

from pvtend.decomposition import compute_orthogonal_basis, project_field
from pvtend.plotting import plot_four_basis, plot_coefficient_curves, plot_field_2d

## 1. Event‑Centred Patch & Base PV Anomaly

40°×40° patch centred on a blocking event. The PV anomaly is a negative Gaussian
whose amplitude evolves over a 10-day lifecycle.

In [ ]:
# Relative coordinates (degrees from event centre)
x_rel = np.arange(-20, 20.1, 1.5)  # ~27 pts lon
y_rel = np.arange(-20, 20.1, 1.5)  # ~27 pts lat
X, Y = np.meshgrid(x_rel, y_rel)
grid_spacing = 1.5  # degrees

# Metres per degree (at ~50°N)
dy_m = np.deg2rad(1.5) * 6.371e6
dx_m = dy_m * np.cos(np.deg2rad(50))

print(f"Patch: {len(y_rel)} × {len(x_rel)} grid points")
print(f"dx ≈ {dx_m/1e3:.0f} km, dy ≈ {dy_m/1e3:.0f} km")

In [ ]:
# Blocking lifecycle: 10-day window, peak at day 5
hours = np.arange(-120, 121, 12)  # -5 days to +5 days, 12-hourly
n_times = len(hours)

# Amplitude envelope: grows, peaks, decays
envelope = np.exp(-hours**2 / (2 * 72**2))  # Gaussian in time, σ = 3 days

# Spatial shape parameters
sigma_x, sigma_y = 8.0, 6.0  # degrees
q_base = -2e-6  # PVU peak amplitude

# Centre drift: eastward at ~5 m/s ≈ 0.055°/h at 50°N
cx_drift = 0.055  # deg/hour
# Slight poleward drift
cy_drift = 0.01  # deg/hour

print(f"Time window: {hours.min()} to {hours.max()} hours, {n_times} steps")

## 2. Build Orthogonal Basis from Peak PV Anomaly

The basis is built once from the composite (peak) PV anomaly and reused for all time steps.

In [ ]:
# Peak PV anomaly (t = 0)
q_peak = q_base * np.exp(-X**2 / (2*sigma_x**2) - Y**2 / (2*sigma_y**2))
dqdx_peak = np.gradient(q_peak, dx_m, axis=1)
dqdy_peak = np.gradient(q_peak, dy_m, axis=0)

basis = compute_orthogonal_basis(
    pv_anom=q_peak,
    pv_dx=dqdx_peak,
    pv_dy=dqdy_peak,
    x_rel=x_rel,
    y_rel=y_rel,
    mask_negative=True,
    apply_smoothing=True,
    smoothing_deg=4.0,
    grid_spacing=grid_spacing,
)

print("Basis norms:", {k: f"{v:.4f}" for k, v in basis.norms.items()})

In [ ]:
# Visualise the four basis fields
fig = plot_four_basis(
    basis.phi_int, basis.phi_dx, basis.phi_dy, basis.phi_def,
    x_rel, y_rel,
    suptitle="Orthogonal Basis (from peak PV anomaly)",
)
plt.show()

## 3. Generate PV Tendency at Each Time Step

At each time step, the PV tendency is computed as the finite-difference $\partial q'/\partial t$,
accounting for:
- **Intensification/decay** (amplitude change)
- **Zonal/meridional propagation** (centre drift)
- **Deformation** (aspect ratio change during decay)

In [ ]:
def make_pv_snapshot(t_hour):
    """Generate PV anomaly at a given time relative to peak."""
    amp = q_base * np.exp(-t_hour**2 / (2 * 72**2))
    cx = cx_drift * t_hour
    cy = cy_drift * t_hour
    # During decay: stretch zonally, compress meridionally → deformation
    sx = sigma_x * (1 + 0.002 * max(t_hour, 0))  # grow after peak
    sy = sigma_y * (1 - 0.001 * max(t_hour, 0))  # shrink after peak
    sy = max(sy, 2.0)  # floor
    return amp * np.exp(-(X - cx)**2 / (2*sx**2) - (Y - cy)**2 / (2*sy**2))


# Compute tendencies via centred finite differences
dt_sec = 12 * 3600  # 12 hours in seconds
tendencies = []
pv_snapshots = [make_pv_snapshot(h) for h in hours]

for i in range(n_times):
    if i == 0:
        dqdt = (pv_snapshots[1] - pv_snapshots[0]) / dt_sec
    elif i == n_times - 1:
        dqdt = (pv_snapshots[-1] - pv_snapshots[-2]) / dt_sec
    else:
        dqdt = (pv_snapshots[i+1] - pv_snapshots[i-1]) / (2 * dt_sec)
    tendencies.append(dqdt)

print(f"Computed {len(tendencies)} tendency fields")
print(f"Max |dq/dt| = {max(np.nanmax(np.abs(t)) for t in tendencies):.2e} PVU/s")

## 4. Project onto Basis → Time Series of β, αx, αy, γ

In [ ]:
coefficients = {"beta": [], "ax": [], "ay": [], "gamma": []}
all_results = []

for i, dqdt in enumerate(tendencies):
    result = project_field(dqdt, basis)
    all_results.append(result)
    for key in coefficients:
        coefficients[key].append(result[key])

for key in coefficients:
    coefficients[key] = np.array(coefficients[key])

print("Peak β  =", f"{coefficients['beta'].max():.3e}  s⁻¹")
print("Peak αx =", f"{coefficients['ax'].max():.3e}  m/s")
print("Peak γ  =", f"{coefficients['gamma'].max():.3e}  s⁻¹")

## 5. Time Evolution Curves (4‑Panel)

The classic lifecycle plot: β > 0 during onset (intensifying), β < 0 during decay;
αx > 0 reflects eastward propagation; γ grows during decay (wave breaking).

In [ ]:
fig = plot_coefficient_curves(
    dh_values=hours,
    coefficients=coefficients,
    title="PV Tendency Decomposition — Blocking Lifecycle",
    xlabel="Hours relative to peak",
)

# Annotate onset / peak / decay
for ax in fig.axes:
    ax.axvspan(-120, -24, alpha=0.05, color='green')
    ax.axvspan(-24, 24, alpha=0.05, color='blue')
    ax.axvspan(24, 120, alpha=0.05, color='red')

# Add stage labels on top axes
fig.axes[0].text(-72, fig.axes[0].get_ylim()[1]*0.85, 'Onset',
                 ha='center', fontsize=9, color='green', fontweight='bold')
fig.axes[0].text(0, fig.axes[0].get_ylim()[1]*0.85, 'Peak',
                 ha='center', fontsize=9, color='blue', fontweight='bold')
fig.axes[0].text(72, fig.axes[0].get_ylim()[1]*0.85, 'Decay',
                 ha='center', fontsize=9, color='red', fontweight='bold')

plt.show()

## 6. 2D Full Fields at Key Lifecycle Stages

Show the reconstructed 2D maps of each component at **onset** (−72 h), **peak** (0 h), and **decay** (+72 h).

In [ ]:
stage_hours = [-72, 0, 72]
stage_labels = ["Onset (−72 h)", "Peak (0 h)", "Decay (+72 h)"]
stage_indices = [np.argmin(np.abs(hours - h)) for h in stage_hours]

# Component labels
component_info = [
    ("int",   r"$\beta \cdot \Phi_1$ — Intensification"),
    ("prop",  r"$\alpha_x \Phi_2 + \alpha_y \Phi_3$ — Propagation"),
    ("def",   r"$\gamma \cdot \Phi_4$ — Wave Breaking"),
    ("resid", "Residual"),
]

fig, axes = plt.subplots(4, 3, figsize=(18, 20))

for col, (si, slabel) in enumerate(zip(stage_indices, stage_labels)):
    res = all_results[si]
    axes[0, col].set_title(slabel, fontsize=13, fontweight='bold')

    for row, (comp_key, comp_label) in enumerate(component_info):
        ax = axes[row, col]
        if comp_key == "prop":
            # Combine propagation: field_prop = -ax_raw * phi_dx - ay_raw * phi_dy
            field = res.get("prop",
                -res["ax_raw"] * basis.phi_dx - res["ay_raw"] * basis.phi_dy)
        elif comp_key == "int":
            field = res.get("int", res["beta_raw"] * basis.phi_int)
        elif comp_key == "def":
            field = res.get("def", res["gamma_raw"] * basis.phi_def)
        else:
            field = res["residual"] if "residual" in res else res.get("resid", np.zeros_like(X))

        vmax = np.nanmax(np.abs(field))
        if vmax < 1e-30:
            vmax = 1.0

        im = ax.imshow(field, origin="lower", cmap="coolwarm",
                        extent=[x_rel.min(), x_rel.max(), y_rel.min(), y_rel.max()],
                        vmin=-vmax, vmax=vmax)
        plt.colorbar(im, ax=ax, shrink=0.75, pad=0.02)
        if col == 0:
            ax.set_ylabel(comp_label, fontsize=10)
        ax.set_xlabel("Δlon (deg)")

plt.suptitle("2D Projected Component Fields across Blocking Lifecycle",
             fontsize=15, y=1.01)
plt.tight_layout()
plt.show()

## 7. Individual 2D Maps: αx, αy, β, γ fields at Peak

Separate panels for each coefficient's contribution at t = 0. These show where
on the PV anomaly each physical process acts.

In [ ]:
res_peak = all_results[np.argmin(np.abs(hours))]

panels = [
    ("Intensification (β·Φ₁)", res_peak.get("int", res_peak["beta_raw"] * basis.phi_int),
     f"β = {res_peak['beta']:.3e} s⁻¹"),
    ("Zonal Propagation (αx·Φ₂)", -res_peak["ax_raw"] * basis.phi_dx,
     f"αx = {res_peak['ax']:.3e} m/s"),
    ("Meridional Propagation (αy·Φ₃)", -res_peak["ay_raw"] * basis.phi_dy,
     f"αy = {res_peak['ay']:.3e} m/s"),
    ("Wave Breaking (γ·Φ₄)", res_peak.get("def", res_peak["gamma_raw"] * basis.phi_def),
     f"γ = {res_peak['gamma']:.3e} s⁻¹"),
]

fig, axes = plt.subplots(2, 2, figsize=(16, 12))

for ax, (title, field, coef_text) in zip(axes.flat, panels):
    vmax = np.nanmax(np.abs(field))
    if vmax < 1e-30:
        vmax = 1.0
    im = ax.imshow(field, origin="lower", cmap="coolwarm",
                    extent=[x_rel.min(), x_rel.max(), y_rel.min(), y_rel.max()],
                    vmin=-vmax, vmax=vmax)
    ax.set_title(f"{title}\n{coef_text}", fontsize=11)
    ax.set_xlabel("Δlon (deg)")
    ax.set_ylabel("Δlat (deg)")
    plt.colorbar(im, ax=ax, shrink=0.8)

plt.suptitle("2D Component Fields at Peak (t = 0)", fontsize=14)
plt.tight_layout()
plt.show()

## 8. Hovmöller Diagrams

Time–longitude Hovmöller of each coefficient to show propagation and lifecycle.

In [ ]:
# Pick the central latitude strip (y_rel ≈ 0) from each component
jc = len(y_rel) // 2  # central latitude index

hov_int = np.array([r.get("int", r["beta_raw"] * basis.phi_int)[jc, :] for r in all_results])
hov_propx = np.array([-r["ax_raw"] * basis.phi_dx[jc, :] for r in all_results])
hov_propy = np.array([-r["ay_raw"] * basis.phi_dy[jc, :] for r in all_results])
hov_def = np.array([r.get("def", r["gamma_raw"] * basis.phi_def)[jc, :] for r in all_results])

fig, axes = plt.subplots(2, 2, figsize=(16, 10))
hovs = [hov_int, hov_propx, hov_propy, hov_def]
titles = ["β·Φ₁ (Intensification)", "αx·Φ₂ (Zonal Prop.)",
          "αy·Φ₃ (Merid. Prop.)", "γ·Φ₄ (Wave Breaking)"]

for ax, data, title in zip(axes.flat, hovs, titles):
    vmax = np.nanmax(np.abs(data))
    if vmax < 1e-30:
        vmax = 1.0
    im = ax.pcolormesh(x_rel, hours, data, cmap="coolwarm",
                        vmin=-vmax, vmax=vmax, shading="auto")
    ax.set_ylabel("Hours relative to peak")
    ax.set_xlabel("Δlon (deg)")
    ax.set_title(title)
    ax.axhline(0, color='k', lw=0.8, ls='--')
    plt.colorbar(im, ax=ax, shrink=0.8)

plt.suptitle("Hovmöller Diagrams (central latitude)", fontsize=14)
plt.tight_layout()
plt.show()

## 9. Reconstruction Quality (RMSE over Time)

In [ ]:
rmse_values = np.array([r["rmse"] for r in all_results])
total_variance = np.array([np.nanstd(t) for t in tendencies])
explained_frac = 1 - (rmse_values / np.maximum(total_variance, 1e-30))**2

fig, axes = plt.subplots(1, 2, figsize=(14, 4))

axes[0].plot(hours, rmse_values, 'ko-', markersize=4)
axes[0].set_xlabel("Hours relative to peak")
axes[0].set_ylabel("RMSE")
axes[0].set_title("Projection RMSE")
axes[0].grid(True, alpha=0.3)

axes[1].plot(hours, explained_frac * 100, 'go-', markersize=4)
axes[1].set_xlabel("Hours relative to peak")
axes[1].set_ylabel("Explained variance (%)")
axes[1].set_title("Reconstruction Quality")
axes[1].set_ylim(0, 105)
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

---

**Summary**:
- **β > 0** during onset (blocking intensifying), **β < 0** during decay
- **αx > 0** consistently = eastward propagation (~5 m/s)
- **αy** shows slight poleward drift
- **γ** grows during the decay phase → wave-breaking / deformation of the PV anomaly
- The 2D fields show **where** on the anomaly each process acts: intensification peaks at the centre,
  propagation shows a dipole pattern, and deformation shows the quadrupole signature